# PER, PBR, PCR, PSR 등 다양한 지표를 통해 종목 선정, 투자하는 기법
- PER: 수익대비주가비율. 주가 / 수익
- PBR: 순자산대비주가비율. 주가 / 순자산
- PCR: 주가현금흐름비율. 시가총액 / 영업현금흐름
- PSR: 주가매출액비율. 시가통액 / 매출액
- ROE: 자기자본대비 얼마나 돈을 잘 버는지
- ROA: 총자산대비 얼마나 돈을 잘 버는지

# 가치주란 ROE, ROA는 좋고 PER, PBR은 낮은 기업을 의미한다. 즉, 자기자본 및 총자산 대비하여 많은 돈을 벌면서 수익이나 순자산에 대비하여 주가는 낮은 기업이 해당하게 된다.

In [4]:
from cmath import nan
import pandas as pd
import datetime
import time
import requests
import numpy as np
import os

In [2]:
dict_div_var = {
    "매출액": "sales_amount",
    "영업이익": "earn",
    "당기순이익": "profit",
    "영업이익률": "earn_rt",
    "순이익률": "profit_rt",
    "ROE(지배주주)": "roe",
    "부채비율": "debt_rt",
    "당좌비율": "check_rt",
    "유보율": "retention_rt",
    "EPS(원)": "eps",
    "PER(배)": "per",
    "BPS(원)": "bps",
    "PBR(배)": "pbr",
    "주당배당금(원)": "devidend_per_stock",
    "시가배당률(%)": "devidend_prc_rt",
    "배당성향(%)": "devidend_rt"
}

In [3]:
def trader_flow_data(cd, pg=1):
    url = f"https://finance.naver.com/item/frgn.naver?code={cd}&page={pg}"
    headers = {
        "referer" : url,    
        "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36" 
    }
    res = requests.get(url=url, headers=headers)      
    df_frgn = pd.read_html(res.text, header=[1])[2].dropna()
    
    df_frgn.rename(columns={"순매매량": "순매매량_기관", "순매매량.1": "순매매량_외국인"}, inplace=True)
    try:
        df_frgn["등락률"] = df_frgn["등락률"].apply(lambda X: float(X.replace("%","").replace("+","")))
        df_frgn["보유율"] = df_frgn["보유율"].apply(lambda X: float(X.replace("%","")))
    except:
        pass
    df_frgn["CD"] = cd

    return df_frgn

In [4]:
def day_sise_flow_data(cd, pg=1):
    url = f"https://finance.naver.com/item/sise_day.naver?code={cd}&page={pg}"
    headers = {
        "referer" : url,    
        "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36" 
    }
    res = requests.get(url=url, headers=headers)
    df_sise = pd.read_html(res.text)[0].dropna().set_index("날짜").reset_index()
    df_sise["CD"] = cd
    
    return df_sise

In [ ]:
def get_bs_data(cd):
    url = f"https://finance.naver.com/item/main.naver?code={cd}"
    try:
        list_df = pd.read_html(url, encoding="euc-kr")
    except Exception as e:
        return None, None, None, None
    
    try:
        forgn_poss_rt = str(list_df[6].iloc[2][1]).strip()
    except Exception as e:
        forgn_poss_rt = None
        
    try:
        list_high_low_prc = list_df[7].iloc[1][1].split("l")
        year_high_prc = str(list_high_low_prc[0]).strip()
        year_low_prc = str(list_high_low_prc[1]).strip()
    except Exception as e:
        year_high_prc = None
        year_low_prc = None
        # print(cd, list_df[7], e)
    
    return list_df[3], year_high_prc, year_low_prc, forgn_poss_rt
    

In [ ]:
def get_day_result(cd):
    url = f"https://finance.naver.com/item/sise.naver?code={cd}"
    list_df = pd.read_html(url, encoding="euc-kr")
    end_prc = list_df[1].iloc[0][1]
    end_vol = list_df[1].iloc[3][1]
    
    return end_prc, end_vol

In [ ]:
dict_bs = {}

def bs_data_to_dict(cd, nm):
    df_bs, year_high_prc, year_low_prc, forgn_poss_rt = get_bs_data(cd)
    if df_bs is None:
        return -1
    
    end_prc, end_vol = get_day_result(cd)
    
    key_idx = 0
    list_index = df_bs.keys()
    try:
        for key in list_index:
            desc = key[0]
            if desc == "주요재무정보": continue
            if desc == "최근 분기 실적": break
            key_idx += 1
        
        dict_value = {}
        dict_value["코드"] = cd
        dict_value["종목명"] = nm
        dict_value["종가"] = int(end_prc)
        dict_value["거래량"] = int(end_vol)
        year_high_prc = int(year_high_prc.replace(",",""))
        dict_value["52주신고가"] = year_high_prc
        year_low_prc = int(year_low_prc.replace(",",""))
        dict_value["52주신저가"] = year_low_prc
        num = pd.Series([year_low_prc, year_high_prc])
        dict_value["사분위75%"] = num.quantile(.75)
        dict_value["사분위50%"] = num.quantile(.5)
        dict_value["사분위25%"] = num.quantile(.25)
        dict_value["외국인지분율"] = float(forgn_poss_rt.replace("%",""))
        for key, row in df_bs.iterrows():
            list_year = []
            list_quarter = []
            for idx in range(len(row)):
                try:
                    dict_div_var[row[idx]]
                    var_nm_year = row[idx] + "_년도"
                    var_nm_quater = row[idx] + "_분기"
                except:
                    val = row[idx]
                    if type(val) == str:
                        val = val.strip().replace("'","").replace("%","")
                    try:
                        val = int(val)
                    except:
                        val = np.nan
                    if idx <= key_idx:
                        list_year.append(val)
                    else:
                        list_quarter.append(val)
            
            dict_value[var_nm_year] = [x for x in list_year if np.isnan(x) == False]                  
            dict_value[var_nm_quater] = [x for x in list_quarter if np.isnan(x) == False]

        dict_bs[cd] = dict_value
    except Exception as e:
        # print(cd, e)
        pass

In [ ]:
# Balace Sheet
df_stock_list = pd.read_csv("stock_list.csv", encoding="utf-8-sig")
# bs_data_to_dict("098120", "SSS")
for row in df_stock_list.values:
    bs_data_to_dict(row[1], row[3])
    
df_bs = pd.DataFrame.from_dict(dict_bs, orient='index')
df_bs["TARGET_TF_ROE"] = df_bs["ROE(지배주주)_년도"].apply(
    lambda X: np.where(np.mean(X) > 15.0, "T", "F")
)
df_bs["TARGET_TF_FRGN"] = df_bs["외국인지분율"].apply(
    lambda X: np.where(float(X) > 5.0, "T", "F")
)
df_bs["HIGH_VS_RT"] = round(df_bs["종가"] / df_bs["52주신고가"], 4) * 100
df_bs["LOW_VS_RT"] = round(df_bs["종가"] / df_bs["52주신저가"], 4) * 100
df_bs["75_VS_RT"] = round(df_bs["종가"] / df_bs["사분위75%"], 4) * 100
df_bs["50_VS_RT"] = round(df_bs["종가"] / df_bs["사분위50%"], 4) * 100
df_bs["25_VS_RT"] = round(df_bs["종가"] / df_bs["사분위25%"], 4) * 100

df_bs

In [ ]:
# ROE 평균 15% 이상이면서 외국인 지분율이 5% 이상이고 사분위수 25%의 85% 아래인 종가 30만원 미만 종목
df_target = df_bs[(df_bs["종가"] < 300000) & (df_bs["TARGET_TF_ROE"] == "T") & (df_bs["TARGET_TF_FRGN"] == "T") & (df_bs["25_VS_RT"] < 85.0)]

In [ ]:
list_target_cd = df_target.sort_values(by=["외국인지분율"], ascending=False)["코드"].unique()
list_target_nm = df_target.sort_values(by=["외국인지분율"], ascending=False)["종목명"].unique()
list_target_cd

In [ ]:
# Company & Foreign Possesion. 외국인 지분율이 상승 중인지 확인을 위함
list_df = []
for cd in list_target_cd:
    list_df.append(trader_flow_data(cd, pg=1))
    
df_trader_flow = pd.concat(list_df)

In [ ]:
list_forgn = []
for cd in list_target_cd:
    now_rt = df_trader_flow[df_trader_flow["CD"] == cd].iloc[0]["보유율"]
    first_rt = df_trader_flow[df_trader_flow["CD"] == cd].iloc[-1]["보유율"]
    updn_val = round(now_rt - first_rt, 2)
    if updn_val > 0.0:
        list_forgn.append(cd)

In [ ]:
def day_sise_flow_data(cd, pg=1):
    url = f"https://finance.naver.com/item/sise_day.naver?code={cd}&page={pg}"
    headers = {
        "referer" : url,    
        "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36" 
    }
    res = requests.get(url=url, headers=headers)
    df_sise = pd.read_html(res.text)[0].dropna().set_index("날짜").reset_index()
    df_sise["CD"] = cd
    
    return df_sise

In [ ]:
# 일별 시세. 올라올 조짐이 있는지 확인을 위함
list_df = []
for cd in list_target_cd:
    list_df.append(day_sise_flow_data(cd, pg=1))
    
df_day_sise_flow = pd.concat(list_df)
df_day_sise_flow.head()

In [ ]:
list_end_prc = []
for cd in list_target_cd:
    now_rt = df_day_sise_flow[df_day_sise_flow["CD"] == cd].iloc[0]["종가"]
    first_rt = df_day_sise_flow[df_day_sise_flow["CD"] == cd].iloc[-1]["종가"]
    updn_val = round(now_rt - first_rt, 2)
    if updn_val > 0.0:
        list_end_prc.append(cd)

In [ ]:
# list(set(list_forgn + list_end_prc))
list_result = list(set(list_forgn) & set(list_end_prc))
list_result

In [ ]:
list_end_prc

In [50]:
import time
import select_quant_jongmok as QUANT
import importlib
importlib.reload(QUANT)

start_logic = time.time()

dict_result = QUANT.execute()

print(f"Elapsed Seconds: {int(time.time() - start_logic)} Sec")

/Users/etlers/src_codes/jupyter/quant_anal/select_quant_jongmok.py:233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target["영업이익_증가"] = df_target["영업이익_분기"].apply(lambda X: check_value_increasing(X))
/Users/etlers/src_codes/jupyter/quant_anal/select_quant_jongmok.py:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target["당기순이익_증가"] = df_target["당기순이익_분기"].apply(lambda X: check_value_increasing(X))


Elapsed Seconds: 691 Sec


In [51]:
dict_result["target_df"].to_csv("bs_target.csv", encoding="utf-8-sig", index=False)

In [53]:
df_target = pd.read_csv("bs_target.csv", encoding="utf-8-sig")
df_target.head()

,코드,종목명,종가,거래량,52주신고가,52주신저가,사분위75%,사분위50%,사분위25%,외국인지분율,...,TARGET_TF_FRGN,HIGH_VS_RT,LOW_VS_RT,75_VS_RT,50_VS_RT,25_VS_RT,PER_MEDIAN,PBR_MEDIAN,영업이익_증가,당기순이익_증가
0,282330,BGF리테일,185500,19983,204000,136500,187125.0,170250.0,153375.0,31.24,...,T,90.93,135.90,99.13,108.96,120.95,18.0,3.0,F,F
1,990,DB하이텍,69900,288604,85400,49200,76350.0,67300.0,58250.0,23.93,...,T,81.85,142.07,91.55,103.86,120.00,10.5,1.5,F,F
2,383220,F&F,141500,132447,199600,86400,171300.0,143000.0,114700.0,13.66,...,T,70.89,163.77,82.60,98.95,123.37,16.5,9.5,F,F
3,51905,LG생활건강우,411500,3333,807000,397500,704625.0,602250.0,499875.0,82.19,...,T,50.99,103.52,58.40,68.33,82.32,25.5,4.0,F,F
4,51900,LG생활건강,720000,54785,1784000,657000,1502250.0,1220500.0,938750.0,37.42,...,T,40.36,109.59,47.93,58.99,76.70,25.5,4.0,F,F


In [6]:
# 영업이익_분기, 당기순이익_분기 두 값이 계속 증가를 했는지 여부

Index(['코드', '종목명', '종가', '거래량', '52주신고가', '52주신저가', '사분위75%', '사분위50%',
       '사분위25%', '외국인지분율', '매출액_년도', '매출액_분기', '영업이익_년도', '영업이익_분기',
       '당기순이익_년도', '당기순이익_분기', '영업이익률_년도', '영업이익률_분기', '순이익률_년도', '순이익률_분기',
       'ROE(지배주주)_년도', 'ROE(지배주주)_분기', '부채비율_년도', '부채비율_분기', '당좌비율_년도',
       '당좌비율_분기', '유보율_년도', '유보율_분기', 'EPS(원)_년도', 'EPS(원)_분기', 'PER(배)_년도',
       'PER(배)_분기', 'BPS(원)_년도', 'BPS(원)_분기', 'PBR(배)_년도', 'PBR(배)_분기',
       '주당배당금(원)_년도', '주당배당금(원)_분기', '시가배당률(%)_년도', '시가배당률(%)_분기',
       '배당성향(%)_년도', '배당성향(%)_분기', 'TARGET_TF_ROE', 'TARGET_TF_FRGN',
       'HIGH_VS_RT', 'LOW_VS_RT', '75_VS_RT', '50_VS_RT', '25_VS_RT',
       'PER_MEDIAN', 'PBR_MEDIAN'],
      dtype='object')

In [41]:
def check_value_increasing(in_val):
    in_val = eval(in_val)
    out_val = "T"
    for i, j in zip(in_val, in_val[1:]):
        if i > j:
            out_val = "F"
            return "F"

    return "T"

In [56]:
df_target["영업이익_증가"] = df_target["영업이익_분기"].apply(lambda X: check_value_increasing(X))
df_target["당기순이익_증가"] = df_target["당기순이익_분기"].apply(lambda X: check_value_increasing(X))
df_target[(df_target["영업이익_증가"] == "T") & (df_target["당기순이익_증가"] == "T")]

,코드,종목명,종가,거래량,52주신고가,52주신저가,사분위75%,사분위50%,사분위25%,외국인지분율,...,TARGET_TF_FRGN,HIGH_VS_RT,LOW_VS_RT,75_VS_RT,50_VS_RT,25_VS_RT,PER_MEDIAN,PBR_MEDIAN,영업이익_증가,당기순이익_증가
1,990,DB하이텍,69900,288604,85400,49200,76350.0,67300.0,58250.0,23.93,...,T,81.85,142.07,91.55,103.86,120.00,10.5,1.5,T,T
35,377450,리파인,12950,57607,18900,11650,17087.5,15275.0,13462.5,5.38,...,T,68.52,111.16,75.79,84.78,96.19,11.0,1.0,T,T
39,60,메리츠화재,39300,111795,53500,17350,44462.5,35425.0,26387.5,11.94,...,T,73.46,226.51,88.39,110.94,148.93,6.0,0.0,T,T


In [29]:
df_jongmok = df_target[df_target["코드"] == 990][["영업이익_분기", "당기순이익_분기", "영업이익_증가", "당기순이익_증가"]]

In [27]:
check_value_increasing([478, 590, 927, 1174, 1587, 1612])

'T'

In [39]:
df_jongmok["영업이익_증가"] = df_jongmok["영업이익_분기"].apply(lambda X: check_value_increasing(X))
df_jongmok["당기순이익_증가"] = df_jongmok["당기순이익_분기"].apply(lambda X: check_value_increasing(X))
df_jongmok

,영업이익_분기,당기순이익_분기,영업이익_증가,당기순이익_증가
1,"[606, 814, 1190, 1381, 1815, 1891]","[478, 590, 927, 1174, 1587, 1612]",T,T
